In [1]:

import sys
sys.path.append('/home/techt/Desktop/a4s')


### Step 1: Initial Data Loading

In [2]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/Chronic_obstructive_pulmonary_disease_(COPD)/GSE212331'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Severity of lung function impairment drives transcriptional phenotypes of COPD and relates to immune and metabolic processes"
!Series_summary	"Gene expression profiles were generated from induced sputum samples in COPD and healthy controls. The study identified transcriptional phenotypes of COPD."
!Series_overall_design	"This study used unsupervised hierarchical clustering of induced sputum gene expression profiles of 72 stable COPD patients and 15 healthy controls to identify distinct and clinically relevant transcriptional COPD phenotypes."
Sample Characteristics Dictionary:
{0: ['tissue: sputum'], 1: ['disease group: COPD', 'disease group: Healthy Control'], 2: ['gold stage: 2', 'gold stage: 3', 'gold stage: 4', 'gold stage: 1', 'gold stage: n/a'], 3: ['age: 75', 'age: 66', 'age: 83', 'age: 70', 'age: 61', 'age: 77', 'age: 64', 'age: 81', 'age: 60', 'age: 62', 'age: 80', 'age: 65', 'age: 74', 'age: 73', 'age: 54', 'age: 67', 'age: 72', 'age: 71

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
is_gene_available = True
trait_row = 1
age_row = 3
gender_row = 4

def convert_trait(value):
    try:
        return 1 if 'COPD' in value.split(':')[1].strip() else 0
    except IndexError:
        return None

def convert_age(value):
    try:
        return float(value.split(':')[1].strip())
    except (IndexError, ValueError):
        return None

def convert_gender(value):
    try:
        return 1 if 'Male' in value.split(':')[1].strip() else 0
    except IndexError:
        return None

save_cohort_info('GSE212331', './preprocessed/Chronic_obstructive_pulmonary_disease_(COPD)/cohort_info.json', is_gene_available, trait_row is not None)

selected_clinical_data = geo_select_clinical_features(clinical_data, 'Chronic_obstructive_pulmonary_disease_(COPD)', trait_row, convert_trait, age_row, convert_age, gender_row, convert_gender)
csv_path = './preprocessed/Chronic_obstructive_pulmonary_disease_(COPD)/trait_data/GSE212331.csv'
selected_clinical_data.to_csv(csv_path)
print(preview_df(selected_clinical_data))


{'GSM6524456': [1.0, 75.0, 0.0], 'GSM6524458': [1.0, 66.0, 1.0], 'GSM6524459': [1.0, 83.0, 1.0], 'GSM6524460': [1.0, 70.0, 0.0], 'GSM6524462': [1.0, 61.0, 0.0], 'GSM6524463': [1.0, 77.0, 1.0], 'GSM6524465': [1.0, 70.0, 0.0], 'GSM6524466': [1.0, 64.0, 0.0], 'GSM6524468': [1.0, 81.0, 1.0], 'GSM6524469': [1.0, 60.0, 0.0], 'GSM6524470': [1.0, 62.0, 0.0], 'GSM6524472': [1.0, 80.0, 1.0], 'GSM6524473': [1.0, 65.0, 0.0], 'GSM6524475': [1.0, 74.0, 1.0], 'GSM6524476': [1.0, 64.0, 1.0], 'GSM6524478': [1.0, 70.0, 0.0], 'GSM6524479': [1.0, 73.0, 1.0], 'GSM6524481': [1.0, 81.0, 1.0], 'GSM6524482': [1.0, 54.0, 1.0], 'GSM6524484': [1.0, 64.0, 1.0], 'GSM6524485': [1.0, 65.0, 1.0], 'GSM6524486': [1.0, 67.0, 1.0], 'GSM6524488': [1.0, 75.0, 1.0], 'GSM6524489': [1.0, 72.0, 0.0], 'GSM6524490': [1.0, 71.0, 1.0], 'GSM6524492': [1.0, 61.0, 0.0], 'GSM6524493': [1.0, 60.0, 1.0], 'GSM6524495': [1.0, 65.0, 1.0], 'GSM6524496': [1.0, 61.0, 0.0], 'GSM6524498': [1.0, 77.0, 0.0], 'GSM6524499': [1.0, 74.0, 1.0], 'GSM652

### Step 3: Gene Data Extraction

In [4]:
# 1. Use the get_genetic_data function from the library to get the gene_data from the matrix_file previously defined.
gene_data = get_genetic_data(matrix_file)

# 2. Print the first 20 row ids for the following step.
print(gene_data.index[:20])


Index(['ILMN_1343291', 'ILMN_1343295', 'ILMN_1651199', 'ILMN_1651209',
       'ILMN_1651210', 'ILMN_1651221', 'ILMN_1651228', 'ILMN_1651229',
       'ILMN_1651230', 'ILMN_1651232', 'ILMN_1651235', 'ILMN_1651236',
       'ILMN_1651237', 'ILMN_1651238', 'ILMN_1651249', 'ILMN_1651253',
       'ILMN_1651254', 'ILMN_1651259', 'ILMN_1651260', 'ILMN_1651262'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
requires_gene_mapping = True


### Step 5: Gene Annotation (Conditional)

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the soft file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['ILMN_1343048', 'ILMN_1343049', 'ILMN_1343050', 'ILMN_1343052', 'ILMN_1343059'], 'Species': [nan, nan, nan, nan, nan], 'Source': [nan, nan, nan, nan, nan], 'Search_Key': [nan, nan, nan, nan, nan], 'Transcript': [nan, nan, nan, nan, nan], 'ILMN_Gene': [nan, nan, nan, nan, nan], 'Source_Reference_ID': [nan, nan, nan, nan, nan], 'RefSeq_ID': [nan, nan, nan, nan, nan], 'Unigene_ID': [nan, nan, nan, nan, nan], 'Entrez_Gene_ID': [nan, nan, nan, nan, nan], 'GI': [nan, nan, nan, nan, nan], 'Accession': [nan, nan, nan, nan, nan], 'Symbol': ['phage_lambda_genome', 'phage_lambda_genome', 'phage_lambda_genome:low', 'phage_lambda_genome:low', 'thrB'], 'Protein_Product': [nan, nan, nan, nan, 'thrB'], 'Probe_Id': [nan, nan, nan, nan, nan], 'Array_Address_Id': [5090180.0, 6510136.0, 7560739.0, 1450438.0, 1240647.0], 'Probe_Type': [nan, nan, nan, nan, nan], 'Probe_Start': [nan, nan, nan, nan, nan], 'SEQUENCE': ['GAATAAAGAACAATCTGCTGATGATCCCTCCGTGGATCTGATTCGTGTAA', 'CCAT

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Identify the identifier key and gene symbol key in annotation
identifier_key = 'ID'
gene_symbol_key = 'Symbol'

# 2. Get the dataframe storing the mapping between probe IDs and genes using the 'get_gene_mapping' function from the library
gene_mapping = get_gene_mapping(gene_annotation, identifier_key, gene_symbol_key)

# 3. Apply the mapping with the 'apply_gene_mapping' function from the library, and name the resulting gene expression dataframe "gene_data".
gene_data = apply_gene_mapping(gene_data, gene_mapping)


### Step 7: Data Normalization and Merging

In [8]:
# 1. Normalize the obtained gene data with the 'normalize_gene_symbols_in_index' function from the library.
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
gene_csv_path = './preprocessed/Chronic_obstructive_pulmonary_disease_(COPD)/gene_data/GSE212331.csv'
normalized_gene_data.to_csv(gene_csv_path)

# 2. Merge the clinical and genetic data with the 'geo_merge_clinical_genetic_data' function from the library.
merged_data = geo_merge_clinical_genetic_data(selected_clinical_data, normalized_gene_data)

# 3. Determine whether the trait and some demographic attributes in the data is severely biased, and remove biased attributes.
trait_biased, unbiased_merged_data = judge_and_remove_biased_features(merged_data, 'Chronic_obstructive_pulmonary_disease_(COPD)')

# 4. Save the cohort information.
save_cohort_info('GSE212331', './preprocessed/Chronic_obstructive_pulmonary_disease_(COPD)/cohort_info.json', True, True, trait_biased, merged_data)

if not trait_biased:
    # 5. If the trait is not severely biased, save the merged data to a csv file.
    csv_path = './preprocessed/Chronic_obstructive_pulmonary_disease_(COPD)/GSE212331.csv'
    unbiased_merged_data.to_csv(csv_path)


For the feature 'Chronic_obstructive_pulmonary_disease_(COPD)', the least common label is '0.0' with 15 occurrences. This represents 17.24% of the dataset.
The distribution of the feature 'Chronic_obstructive_pulmonary_disease_(COPD)' in this dataset is fine.

Quartiles for 'Age':
  25%: 62.0
  50% (Median): 69.0
  75%: 74.5
Min: 20.0
Max: 88.0
The distribution of the feature 'Age' in this dataset is fine.

For the feature 'Gender', the least common label is '0.0' with 41 occurrences. This represents 47.13% of the dataset.
The distribution of the feature 'Gender' in this dataset is fine.

